In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

import datetime
import time

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}

npr_title_new = []
npr_time = []
section = ['world', 'national', 'politics', 'health', 'technology', 'business', 'science', 'race', 'codeswitch', 'law']
months = [
    {'month': 1, 'last': 31},
    {'month': 2, 'last': 28},
    {'month': 3, 'last': 31},
    {'month': 4, 'last': 30},
    {'month': 5, 'last': 31},
    {'month': 6, 'last': 30},
    {'month': 7, 'last': 31},
    {'month': 8, 'last': 31},
    {'month': 9, 'last': 30},
    {'month': 10, 'last': 31},
    {'month': 11, 'last': 30},
    {'month': 12, 'last': 31},
]

for s in section:
    for year in range(2019, 2021):
        for entry in months:
            month = entry['month']
            for day in range(1, entry['last'] + 1):
                res = requests.get(f'https://www.npr.org/sections/{s}/archive?date={month}-{day}-{year}', headers = headers)
                each_page = BeautifulSoup(res.text, 'html.parser')
                title = each_page.find_all(class_='title')
                time = each_page.find_all(class_='date')

                all_titles=[]
                for e in title:
                    all_titles.append(e.get_text())

                all_time=[]
                for t in time:
                    t_change = t.get_text()[:-3].replace(',', '')
                    t_change = t_change.replace(' ', '-')
        #            all_time.append(datetime.datetime.strptime(t_change, '%B-%d-%y').strftime('%m-%d-%y'))
                    all_time.append(t_change)       

                npr_title_new.append(all_titles)
                npr_time.append(all_time)



In [ ]:
for s in section:
    for year in range(2016, 2021):
        for month in months['month']:
            for day in months:
                res = requests.get(f'https://www.npr.org/sections/{s}/archive?date={month}-{day}-{year}', headers = headers)
                each_page = BeautifulSoup(res.text, 'html.parser')
                title = each_page.find_all(class_='title')
                time = each_page.find_all(class_='date')

                all_titles=[]
                for e in title:
                    all_titles.append(e.get_text())

                all_time=[]
                for t in time:
                    t_change = t.get_text()[:-3].replace(',', '')
                    t_change = t_change.replace(' ', '-')
        #            all_time.append(datetime.datetime.strptime(t_change, '%B-%d-%y').strftime('%m-%d-%y'))
                    all_time.append(t_change)       

                npr_title.append(all_titles)
                npr_time.append(all_time)
    

In [5]:
npr_title_clean = []
for lst in npr_title:
    for j in lst:
        npr_title_clean.append(j)

npr_time_clean = []
for lst in npr_time:
    for j in lst:
        npr_time_clean.append(j)
np.shape(npr_title_clean)

(15822,)

In [8]:
dictionary = {'title':npr_title_clean, 'date':npr_time_clean} 
npr = pd.DataFrame(dictionary)
npr

,title,date
0,Hotel Fire Raises Concerns About Safety Of Dub...,January-1-2016
1,Blind Masseur Helps Relieve Tension In War-Tor...,January-1-2016
2,Smoke Still Rising From Dubai High-Rise Amid F...,January-1-2016
3,Gunman In Downtown Tel Aviv Kills 2 Israelis,January-1-2016
4,New Year's Eve Fireworks In Dubai Go On Despit...,January-1-2016
...,...,...
15817,Tentative Brexit Deal Keeps Border Porous Betw...,November-21-2018
15818,"News Brief: Trump Defends Saudi Arabia, Miss. ...",November-21-2018
15819,What Will Persuade Rice Farmers In Punjab To S...,November-21-2018
15820,Bangladeshi Photojournalist And Activist Freed...,November-21-2018


In [ ]:
npr['source'] = 'npr'
npr['is_sarcastic'] = 0
npr['date'] = npr['date'].apply(lambda x: datetime.datetime.strptime(x, '%B-%d-%y').strftime('%Y-%m-%d'))


In [7]:
npr['timestamp'] = 0
for i in range(len(npr['date'])):
    npr['timestamp'][i] = time.mktime(datetime.datetime.strptime(npr['date'][i], '%B-%d-%Y').timetuple())


AttributeError: ResultSet object has no attribute 'mktime'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [43]:
npr = pd.read_csv('NPR 2016 to 2020.csv')
npr = npr.sort_values(by=['timestamp'])
npr = npr.set_index(np.arange(0, len(npr['timestamp']), 1))

npr_drop2015 = npr[6:]
npr_unique = npr_drop2015.drop_duplicates()

In [56]:
npr_unique.to_csv('NPR 2016 to 2020.csv', index = False)

In [57]:
npr_unique

,title,date,timestamp,source
6,Smoke Still Rising From Dubai High-Rise Amid F...,January-1-2016,1451624400,npr
7,Gunman In Downtown Tel Aviv Kills 2 Israelis,January-1-2016,1451624400,npr
8,New Year's Eve Fireworks In Dubai Go On Despit...,January-1-2016,1451624400,npr
9,Brazilians Welcome The New Year At Big Beach P...,January-1-2016,1451624400,npr
10,Hotel Fire Raises Concerns About Safety Of Dub...,January-1-2016,1451624400,npr
...,...,...,...,...
17361,Coronavirus World Map: Tracking The Spread Of ...,November-5-2020,1604552400,npr
17363,Brexit Seems To Benefit The Tiny Nation Of Lux...,November-5-2020,1604552400,npr
17364,Kosovo's President Steps Down To Face War Crim...,November-5-2020,1604552400,npr
17365,Poor Countries Fall Behind In Race To Reserve ...,November-5-2020,1604552400,npr
